In [6]:
# !pip install selenium
# !pip install beautifulsoup4

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

In [4]:
def driver_function(url):
    driver = webdriver.Chrome("/usr/local/share/chromedriver")
    driver.get(url)
    content = driver.page_source
    driver.quit()
    return content

In [5]:
adoption_links = []
for i in range(1,5):
    url = f"https://sosd.org.sg/adopt-a-dog/{i}/?dog&gender&hdb"
    soup = BeautifulSoup(driver_function(url), 'html.parser')
    
    links = soup.findAll('a')
    for link in links:
        result = link.get('href')
        if "wpcproduct" in result and result not in adoption_links:
            adoption_links.append(result)

print(adoption_links)
    

/tmp/ipykernel_3025/1116036059.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/local/share/chromedriver")


['https://sosd.org.sg/wpcproduct/benji/?form=adopt', 'https://sosd.org.sg/wpcproduct/buddy/?form=adopt', 'https://sosd.org.sg/wpcproduct/cadbury/?form=adopt', 'https://sosd.org.sg/wpcproduct/carter/?form=adopt', 'https://sosd.org.sg/wpcproduct/cody/?form=adopt', 'https://sosd.org.sg/wpcproduct/cooper/?form=adopt', 'https://sosd.org.sg/wpcproduct/damien/?form=adopt', 'https://sosd.org.sg/wpcproduct/debra/?form=adopt', 'https://sosd.org.sg/wpcproduct/dusty/?form=adopt', 'https://sosd.org.sg/wpcproduct/eli/?form=adopt', 'https://sosd.org.sg/wpcproduct/godiva/?form=adopt', 'https://sosd.org.sg/wpcproduct/hooch/?form=adopt', 'https://sosd.org.sg/wpcproduct/jacky/?form=adopt', 'https://sosd.org.sg/wpcproduct/jerry/?form=adopt', 'https://sosd.org.sg/wpcproduct/kaiser/?form=adopt', 'https://sosd.org.sg/wpcproduct/kiki/?form=adopt', 'https://sosd.org.sg/wpcproduct/koko/?form=adopt', 'https://sosd.org.sg/wpcproduct/latte/?form=adopt', 'https://sosd.org.sg/wpcproduct/libby/?form=adopt', 'https://

In [42]:
dog_adoption_org = []
dog_adopt_link = []
dog_names = []
dog_gender = []
dog_dob_est = []
dog_personality = []

for adoption_link in adoption_links:
    soup = BeautifulSoup(driver_function(adoption_link))
    data_extract = soup.findAll('div', href=False, attrs={'class':"entry-content"})
    html_removed = re.sub(r'<.*?>', '', str(data_extract))
    html_removed = html_removed.replace("\n","")
    html_removed = html_removed.replace("[","")
    html_removed = html_removed.replace("]","")

    #Organisation data
    dog_adoption_org.append("SOSD")
    dog_adopt_link.append(adoption_link)
    
    #Name data
    if "Sponsored" in html_removed:
        name = html_removed[:html_removed.index("Sponsored")]
        dog_names.append(name)

    else:
        name = html_removed[:html_removed.index("Be my sponsor")]
        dog_names.append(name)
        
    #Gender data
    gender_data = html_removed[html_removed.index("Gender")+8:html_removed.index("HDB")]
    dog_gender.append(gender_data)

    #DOB data
    dob_data = html_removed[html_removed.index("Estimated")+15:html_removed.index("Personality")]
    dog_dob_est.append(dob_data)

    #Personality data
    personality_data = html_removed[html_removed.index("Personality")+13:html_removed.index("If you are keen to share your home and love with")]
    dog_personality.append(personality_data)

print(dog_names[:2])
print(dog_gender[:2])
print(dog_dob_est[:2])
print(dog_personality[:2])

/tmp/ipykernel_3025/1116036059.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/local/share/chromedriver")


['Benji', 'Buddy']
['Male', 'Male']
['1 Apr 2011', '7 Sep 2016']
['You want to take me out for a walk? If you do, you would be my favourite hooman. 😊Hi, I’m Benji! I love going for walks, anytime, any day. It’s my favourite thing to do. Once I hear or see you taking the leash out, I would do a little twirly whirly dance with my helicopter tail going at 200km/h.I’m a good boi and I understand simple commands like sit, come and wait. I know good manners come with nice rewards. You know what to do right? Treats are my friends of course.Be patient with me, I get spooked easily and am wary of people if they raise their voice or shout, even dramatic hand gestures or postures. I might growl because I’m scared. If you can assure me everything is OK, I’ll calm down rather quickly too.I think I am 10 years old, but, age is just a number you know. I will always like tummy rubs, hugs, cuddles, sitting on your lap if you let me. Get me a bone and I’ll be happily carrying it around in the house. I n

In [43]:
len(dog_personality)

45

In [44]:
df = pd.DataFrame({'Organisation':dog_adoption_org,'Link':dog_adopt_link,'Name':dog_names,'Gender':dog_gender,'Estimated DOB':dog_dob_est,"Personality":dog_personality}) 
df.to_excel('Dogs_SOSD.xlsx', index=False)